In [1]:
#!pip install newsapi-python

In [2]:
#!pip install python-dotenv

In [3]:
# Initial imports
import os
import pandas as pd
from newsapi import NewsApiClient
import nltk as nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\srava\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [6]:
# Read your api key environment variable
# YOUR CODE HERE!
api_key = os.getenv('news_api')
type(api_key)

str

In [7]:
# Create a newsapi client
# YOUR CODE HERE!
newsapi = NewsApiClient(api_key=api_key)
#type(newsapi)

In [8]:
# Fetch the ESG news articles
# YOUR CODE HERE!
ESG_headlines = newsapi.get_everything(q='ESG',
                                          language='en',
                                          page_size=100,
                                          sort_by='relevancy'
                                          )
print(f"Total articles about ESG: {ESG_headlines['totalResults']}")

ESG_headlines["articles"][0]

Total articles about ESG: 2477


{'source': {'id': 'reuters', 'name': 'Reuters'},
 'author': None,
 'title': 'DLA Piper debuts new executive level sustainability role - Reuters',
 'description': 'As more law firms go beyond establishing environmental, social and governance practices and embrace internal ESG initiatives, DLA Piper announced Tuesday that it had created a new executive-level ESG leadership role to spearhead ESG efforts across its interna…',
 'url': 'https://www.reuters.com/legal/litigation/dla-piper-debuts-new-executive-level-sustainability-role-2021-09-07/',
 'urlToImage': 'https://www.reuters.com/resizer/WEV3KYG5a_rrxFB6mmPjaVPTojY=/1200x628/smart/filters:quality(80)/cloudfront-us-east-2.images.arcpublishing.com/reuters/WDOFVP3KOVLEXAUOUKU3P5PWTI.jpg',
 'publishedAt': '2021-09-07T22:45:00Z',
 'content': 'DLA Piper offices in Washington, D.C. REUTERS/Andrew KellyThe company and law firm names shown above are generated automatically based on the text of the article. We are improving this feature as we … 

In [42]:
# Create the ESG sentiment scores DataFrame
# YOUR CODE HERE!
ESG_sentiments = []

for article in ESG_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        ESG_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ESG_df = pd.DataFrame(ESG_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
ESG_df = ESG_df[cols]

ESG_df.head(100)

,date,text,compound,positive,negative,neutral
0,2021-09-07T22:45:00Z,"DLA Piper offices in Washington, D.C. REUTERS/...",0.4215,0.078,0.0,0.922
1,2021-08-26T13:34:00Z,(Reuters) - India has fewer funds focused on e...,0.5267,0.124,0.0,0.876
2,2021-09-08T17:56:00Z,A view of the exterior of the JP Morgan Chase ...,0.3680,0.073,0.0,0.927
3,2021-08-26T13:41:00Z,Aug 26 (Reuters) - India has fewer funds focus...,0.5267,0.118,0.0,0.882
4,2021-08-19T01:34:00Z,"HONG KONG, Aug 19 (Reuters) - Chinese internet...",0.0000,0.000,0.0,1.000
...,...,...,...,...,...,...
95,2021-09-01T09:33:00Z,Cathie Woods ARK Investment Management is plan...,0.3818,0.073,0.0,0.927
96,2021-08-28T21:17:14Z,Close-up of logo for industrial gas supplier L...,0.2500,0.062,0.0,0.938
97,2021-08-24T18:19:57Z,SynopsisAs ESG financing gains traction across...,0.6249,0.145,0.0,0.855
98,2021-08-22T15:00:02Z,Investors and politicians embracing a vision o...,0.6486,0.189,0.0,0.811


In [10]:
#Adding a label to analyze the compound value. A score greater than 0.2 is positive and a score less than -0.2 is negative

ESG_df['label'] = 0
ESG_df.loc[ESG_df['compound'] > 0.2, 'label'] = 1
ESG_df.loc[ESG_df['compound'] < -0.2, 'label'] = -1
ESG_df.head()

,date,text,compound,positive,negative,neutral,label
0,2021-09-07T22:45:00Z,DLA Piper debuts new executive level sustainab...,0.0000,0.000,0.000,1.000,0
1,2021-08-26T13:34:00Z,GRAPHIC-India has fewer ESG funds than other t...,0.2023,0.153,0.000,0.847,1
2,2021-09-08T17:56:00Z,MOVES JPMorgan names new head of ESG for inves...,0.0000,0.000,0.000,1.000,0
3,2021-08-26T13:41:00Z,India has fewer ESG funds than other top 10 ec...,0.2023,0.153,0.000,0.847,1
4,2021-08-19T01:34:00Z,Baidu raises $1 bln in ESG bond issue despite ...,-0.4404,0.000,0.182,0.818,-1


In [11]:
#printing the results of above analysis. there are 50 articles that are considered "Neutral", 
#36 articles that are "positive" and 14 articles that are "negative"

print(ESG_df.label.value_counts())



 0    49
 1    37
-1    14
Name: label, dtype: int64


In [12]:
ESG_df.describe()


,compound,positive,negative,neutral,label
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.118585,0.108230,0.036930,0.854830,0.230000
std,0.313731,0.150629,0.084217,0.152649,0.679498
min,-0.571900,0.000000,0.000000,0.446000,-1.000000
25%,0.000000,0.000000,0.000000,0.757500,0.000000
50%,0.000000,0.000000,0.000000,0.847000,0.000000
75%,0.318200,0.204250,0.000000,1.000000,1.000000
max,0.812600,0.554000,0.419000,1.000000,1.000000


In [13]:
CO2_headlines = newsapi.get_everything(q='CO2',
                                          language='en',
                                          page_size=100,
                                          sort_by='relevancy'
                                          )
print(f"Total articles about CO2: {CO2_headlines['totalResults']}")

CO2_headlines["articles"][0]

Total articles about CO2: 3313


{'source': {'id': 'techcrunch', 'name': 'TechCrunch'},
 'author': 'Devin Coldewey',
 'title': 'Heimdal pulls CO2 and cement-making materials out of seawater using renewable energy',
 'description': 'One of the consequences of rising CO2 levels in our atmosphere is that levels also rise proportionately in the ocean, harming wildlife and changing ecosystems. Heimdal is a startup working to pull that CO2 back out at scale using renewable energy and producin…',
 'url': 'http://techcrunch.com/2021/08/30/heimdal-pulls-co2-and-cement-making-materials-out-of-seawater-using-renewable-energy/',
 'urlToImage': 'https://techcrunch.com/wp-content/uploads/2021/08/heimdal-header.jpg?w=764',
 'publishedAt': '2021-08-30T16:33:37Z',
 'content': 'One of the consequences of rising CO2 levels in our atmosphere is that levels also rise proportionately in the ocean, harming wildlife and changing ecosystems. Heimdal is a startup working to pull th… [+6484 chars]'}

In [43]:
# Create the CO2 sentiment scores DataFrame
CO2_sentiments = []

for article in CO2_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        CO2_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
CO2_df = pd.DataFrame(CO2_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
CO2_df = CO2_df[cols]

CO2_df.head()

,date,text,compound,positive,negative,neutral
0,2021-08-30T16:33:37Z,One of the consequences of rising CO2 levels i...,-0.5574,0.000,0.098,0.902
1,2021-09-09T18:55:56Z,The world's largest direct-air carbon capture ...,0.0772,0.042,0.000,0.958
2,2021-09-10T15:27:00Z,"OSLO, Sept 10 (Reuters) - Norway is offering t...",0.5859,0.127,0.000,0.873
3,2021-09-13T12:30:00Z,Scientists with the University of Victoria-bas...,0.1779,0.059,0.000,0.941
4,2021-09-13T12:30:00Z,Scientists with the University of Victoria-bas...,0.1779,0.059,0.000,0.941


In [44]:
#Adding a label to analyze the compound value. A score greater than 0.2 is positive and a score less than -0.2 is negative

CO2_df['label'] = 0
CO2_df.loc[CO2_df['compound'] > 0.2, 'label'] = 1
CO2_df.loc[CO2_df['compound'] < -0.2, 'label'] = -1
CO2_df.head()

,date,text,compound,positive,negative,neutral,label
0,2021-08-30T16:33:37Z,One of the consequences of rising CO2 levels i...,-0.5574,0.000,0.098,0.902,-1
1,2021-09-09T18:55:56Z,The world's largest direct-air carbon capture ...,0.0772,0.042,0.000,0.958,0
2,2021-09-10T15:27:00Z,"OSLO, Sept 10 (Reuters) - Norway is offering t...",0.5859,0.127,0.000,0.873,1
3,2021-09-13T12:30:00Z,Scientists with the University of Victoria-bas...,0.1779,0.059,0.000,0.941,0
4,2021-09-13T12:30:00Z,Scientists with the University of Victoria-bas...,0.1779,0.059,0.000,0.941,0


In [45]:
#printing the results of above analysis. there are 63 articles that are considered "Neutral", 
#27 articles that are "positive" and 10 articles that are "negative"

print(CO2_df.label.value_counts())


 1    40
 0    39
-1    21
Name: label, dtype: int64


In [46]:
CO2_df.describe()


,compound,positive,negative,neutral,label
count,100.000000,100.000000,100.00000,100.000000,100.000000
mean,0.114931,0.060280,0.03022,0.909480,0.190000
std,0.395524,0.071261,0.04572,0.075067,0.761378
min,-0.812600,0.000000,0.00000,0.668000,-1.000000
25%,0.000000,0.000000,0.00000,0.858000,0.000000
50%,0.012900,0.044000,0.00000,0.922000,0.000000
75%,0.440400,0.102250,0.06525,1.000000,1.000000
max,0.897900,0.332000,0.16100,1.000000,1.000000


In [47]:
CO2_emissions_headlines = newsapi.get_everything(q='CO2 Emissions',
                                          language='en',
                                          page_size=100,
                                          sort_by='relevancy'
                                          )
print(f"Total articles about CO2: {CO2_headlines['totalResults']}")

CO2_emissions_headlines["articles"][0]

Total articles about CO2: 3313


{'source': {'id': 'engadget', 'name': 'Engadget'},
 'author': 'Jon Fingas',
 'title': 'The largest direct carbon capture plant is now making a small impact on emissions',
 'description': "The world's largest direct-air carbon capture plant is up and running, although you might want to keep your expectations in check. Bloombergreports Climeworks has started operations for Orca, a plant in Iceland that grabs CO2 from the air, storing it undergro…",
 'url': 'https://www.engadget.com/worlds-largest-carbon-capture-plant-185556109.html',
 'urlToImage': 'https://s.yimg.com/os/creatr-uploaded-images/2021-09/2b974d80-1198-11ec-9bfc-8ba21f637771',
 'publishedAt': '2021-09-09T18:55:56Z',
 'content': "The world's largest direct-air carbon capture plant is up and running, although you might want to keep your expectations in check. Bloombergreports Climeworks has started operations for Orca, a plant… [+1474 chars]"}

In [48]:
# Create the CO2 sentiment scores DataFrame
CO2_emissions_sentiments = []

for article in CO2_emissions_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        CO2_emissions_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
CO2_emissions_df = pd.DataFrame(CO2_emissions_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
CO2_emissions_df = CO2_emissions_df[cols]

CO2_emissions_df.head()

,date,text,compound,positive,negative,neutral
0,2021-09-09T18:55:56Z,The world's largest direct-air carbon capture ...,0.0772,0.042,0.000,0.958
1,2021-08-23T18:11:16Z,Y Combinator-backed Kapacity.io is on a missio...,0.7003,0.195,0.000,0.805
2,2021-08-22T15:00:02Z,Investors and politicians embracing a vision o...,0.6486,0.189,0.000,0.811
3,2021-08-30T16:33:37Z,One of the consequences of rising CO2 levels i...,-0.5574,0.000,0.098,0.902
4,2021-08-25T17:00:27Z,Renewables arent growing fast enough \r\nWind ...,-0.1326,0.000,0.043,0.957


In [49]:
#Adding a label to analyze the compound value. A score greater than 0.2 is positive and a score less than -0.2 is negative

CO2_emissions_df['label'] = 0
CO2_emissions_df.loc[CO2_emissions_df['compound'] > 0.2, 'label'] = 1
CO2_emissions_df.loc[CO2_emissions_df['compound'] < -0.2, 'label'] = -1
CO2_emissions_df.head()

,date,text,compound,positive,negative,neutral,label
0,2021-09-09T18:55:56Z,The world's largest direct-air carbon capture ...,0.0772,0.042,0.000,0.958,0
1,2021-08-23T18:11:16Z,Y Combinator-backed Kapacity.io is on a missio...,0.7003,0.195,0.000,0.805,1
2,2021-08-22T15:00:02Z,Investors and politicians embracing a vision o...,0.6486,0.189,0.000,0.811,1
3,2021-08-30T16:33:37Z,One of the consequences of rising CO2 levels i...,-0.5574,0.000,0.098,0.902,-1
4,2021-08-25T17:00:27Z,Renewables arent growing fast enough \r\nWind ...,-0.1326,0.000,0.043,0.957,0


In [50]:
#printing the results of above analysis. there are 63 articles that are considered "Neutral", 
#27 articles that are "positive" and 10 articles that are "negative"

print(CO2_emissions_df.label.value_counts())


 1    42
 0    34
-1    24
Name: label, dtype: int64


In [51]:
CO2_emissions_df.describe()


,compound,positive,negative,neutral,label
count,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.106043,0.068440,0.037470,0.894080,0.180000
std,0.423814,0.072167,0.052013,0.080514,0.796203
min,-0.812600,0.000000,0.000000,0.668000,-1.000000
25%,-0.110175,0.000000,0.000000,0.846750,0.000000
50%,0.101850,0.058000,0.000000,0.904500,0.000000
75%,0.440400,0.127250,0.071250,0.944250,1.000000
max,0.897900,0.332000,0.187000,1.000000,1.000000


In [52]:
#!pip install spacy

In [53]:
#!python -m spacy download en_core_web_lg

In [54]:
import spacy
from spacy import displacy

In [55]:
nlp = spacy.load('en_core_web_sm')

In [56]:
flat_esg_text_str = " ".join(ESG_df.text.to_list())
print(flat_esg_text_str)

DLA Piper offices in Washington, D.C. REUTERS/Andrew KellyThe company and law firm names shown above are generated automatically based on the text of the article. We are improving this feature as we … [+3424 chars] (Reuters) - India has fewer funds focused on environmental, social and governance (ESG) issues than other top 10 economies, data shows, amid investor caution about ESG funds that have yet to build a … [+2510 chars] A view of the exterior of the JP Morgan Chase &amp; Co corporate headquarters in New York City May 20, 2015. REUTERS/Mike Segar/File PhotoNEW YORK, Sept 8 (Reuters) - JPMorgan Chase &amp; Co (JPM.N) … [+873 chars] Aug 26 (Reuters) - India has fewer funds focused on environmental, social and governance (ESG) issues than other top 10 economies, data shows, amid investor caution about ESG funds that have yet to b… [+2476 chars] HONG KONG, Aug 19 (Reuters) - Chinese internet giant Baidu Inc has raised $1 billion in a two-tranche, U.S. dollar sustainability bond, its f

In [57]:
# Run the NER processor on all of the text
ESG_doc = nlp(flat_esg_text_str)
ESG_doc.user_data["title"] = "ESG NER"
# Add a title to the document

In [58]:

displacy.render(ESG_doc,style='ent')

In [59]:
# List all Entities
for ent in ESG_doc.ents:
    print(ent.text, ent.label_)

DLA Piper ORG
Washington GPE
D.C. GPE
Andrew KellyThe PERSON
Reuters ORG
India GPE
10 CARDINAL
Morgan Chase &amp ORG
New York City GPE
May 20, 2015 DATE
REUTERS ORG
Mike Segar PERSON
Sept 8 DATE
Reuters ORG
JPMorgan Chase &amp ORG
Co PERSON
26 CARDINAL
Reuters ORG
India GPE
10 CARDINAL
HONG KONG GPE
Aug 19 DATE
Reuters ORG
Chinese NORP
Baidu Inc ORG
$1 billion MONEY
two CARDINAL
U.S. GPE
first ORDINAL
Reuters ORG
FILE ORG
Graaff-Reinet PERSON
South Africa GPE
November 17, 2019 DATE
November 17, 2019 DATE
Aug 31 DATE
Reuters ORG
Franklin Templeton PERSON
Tuesday DATE
U.S. GPE
1977 DATE
Palm Desert GPE
California GPE
U.S. GPE
June 29, 2021 DATE
REUTERS ORG
Reuters ORG
more than 840,000 CARDINAL
today DATE
$5 million MONEY
UBS Investment Bank ORG
HONG KONG GPE
Aug 19 DATE
Reuters ORG
Chinese NORP
Baidu Inc (9888.HK ORG
$1 billion MONEY
two CARDINAL
U.S. GPE
first ORDINAL
the Saudi Public Investment Fund Yasir Othman Al-Rumayyan ORG
Saudi NORP
Mohammed Abdullah Al Jadaan PERSON
Ahmed Aqeel

In [60]:
flat_CO2_text_str = " ".join(CO2_df.text.to_list())
print(flat_CO2_text_str)

One of the consequences of rising CO2 levels in our atmosphere is that levels also rise proportionately in the ocean, harming wildlife and changing ecosystems. Heimdal is a startup working to pull th… [+6484 chars] The world's largest direct-air carbon capture plant is up and running, although you might want to keep your expectations in check. Bloombergreports Climeworks has started operations for Orca, a plant… [+1474 chars] OSLO, Sept 10 (Reuters) - Norway is offering two new offshore areas for companies interested in developing carbon dioxide storage, the oil and energy ministry said on Friday.
One of the areas is eas… [+1035 chars] Scientists with the University of Victoria-based Pacific Institute for Climate Solutions believe there is a possibility to sequester carbon dioxide in the subseafloor ocean basalt.
Carbon sequestrat… [+1316 chars] Scientists with the University of Victoria-based Pacific Institute for Climate Solutions believe there is a possibility to sequester carbon di

In [61]:
# Run the NER processor on all of the text
CO2_doc = nlp(flat_CO2_text_str)
CO2_doc.user_data["title"] = "CO2 NER"
# Add a title to the document
displacy.render(CO2_doc,style='ent')

In [62]:
for ent in co2_doc.ents:
    print(ent.text, ent.label_)

Norway GPE
two CARDINAL
U-Calgary ORG
U-Calgary ORG
Peninsula News Review - Peninsula News Review Zurich Insurance ORG
South Africa GPE
2023 CARDINAL
AI ORG
Hago Energetics Inc. ORG
NASA CO2 Conversion Challenge Award - Digital Journal World ORG
Iceland GPE
China GPE
CNOOC NORP
first ORDINAL
G-Force ORG
Chains PERSON
Earth LOC
Earth LOC
Yahoo News Canada Tata ORG
Dutch NORP
Iceland GPE
Indonesia GPE
Ubadari ORG
Vorwata GPE
Reuters ORG
Germany GPE
Reuters ORG
95% PERCENT
2100 DATE
Yahoo News Canada ArcelorMittal ORG
Hamburg GPE
Germany GPE
Germany GPE
first ORDINAL
EU ORG
Russia GPE
zero CARDINAL
U.N. ORG
England GPE
Covid Record PERSON
EU ORG
Hamburg GPE
UK GPE
Ferrari GPE
Hebridean NORP
zero CARDINAL
Covid PERSON
Swedish NORP
Earth LOC
CO2 Ozone Hole Would PERSON
95% PERCENT
Surface Ocean Conditions - News@Northeastern Norway's ORG
Troll ORG
Reuters ORG
US GPE
EU ORG
zero CARDINAL
2050 DATE
Kenya GPE
2 CARDINAL
Reuters ORG
British NORP
40% PERCENT
2030 CARDINAL
1 CARDINAL
Norway GPE
R

In [63]:
flat_co2_emissions_text_str = " ".join(CO2_emissions_df.text.to_list())
print(flat_co2_emissions_text_str)

The world's largest direct-air carbon capture plant is up and running, although you might want to keep your expectations in check. Bloombergreports Climeworks has started operations for Orca, a plant… [+1474 chars] Y Combinator-backed Kapacity.io is on a mission to accelerate the decarbonization of buildings by using AI-generated efficiency savings to encourage electrification of commercial real estate — wooing… [+15270 chars] Investors and politicians embracing a vision of an all-electric car future believe that path will significantly reduce global carbon dioxide emissions. Thats far from clear.
A growing body of resear… [+22132 chars] One of the consequences of rising CO2 levels in our atmosphere is that levels also rise proportionately in the ocean, harming wildlife and changing ecosystems. Heimdal is a startup working to pull th… [+6484 chars] Renewables arent growing fast enough 
Wind turbines in front of a coal-fired power plant on the outskirts of the new city area of Yumen, Ga

In [64]:
# Run the NER processor on all of the text
CO2_emissions_doc = nlp(flat_co2_emissions_text_str)
CO2_emissions_doc.user_data["title"] = "CO2 Emissions NER"
# Add a title to the document
displacy.render(CO2_emissions_doc,style='ent')

In [65]:
for ent in CO2_emissions_doc.ents:
    print(ent.text, ent.label_)

Orca ORG
Kapacity.io PRODUCT
AI ORG
One CARDINAL
Renewables PERSON
Yumen GPE
Gansu province GPE
China GPE
Wednesday, March 31, 2021 DATE
Wind tu PERSON
Mario Greco PERSON
Swiss Zurich Insurance ORG
Zurich GPE
Switzerland GPE
February 9, 2017 DATE
REUTERS ORG
Arnd Wiegmann PERSON
Sept 7 DATE
Reuters ORG
ZURN ORG
Reuters ORG
Sept 7 DATE
Reuters ORG
Swiss NORP
Reuters ORG
FILE ORG
Mario Greco PERSON
Swiss Zurich Insurance ORG
Zurich GPE
Switzerland GPE
February 9, 2017 DATE
REUTERS ORG
Arnd Wiegmann PERSON
File Photo
ZURICH ORG
Reuters ORG
the University of Victoria-based ORG
Pacific Institute for Climate Solutions ORG
the University of Victoria-based ORG
Pacific Institute for Climate Solutions ORG
FILE ORG
Matautu PERSON
Japan GPE
Samoan ORG
Apia GPE
July 12, 2019 DATE
July 12, 2019 DATE
Sept 10 DATE
Reuters ORG
Norway GPE
two CARDINAL
energy ministry ORG
Friday DATE
One CARDINAL
Mexican NORP
U.S. GPE
Otay Mesa PERSON
California GPE
September 10, 2013 DATE
REUTERS ORG
Mike Blake/ZURICH P